<h1 style="text-align:center;">Configuring the ESKM key for HPE iLO systems</h1>

In [ ]:
<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This script can be used to manage and configure the ESKM key server for HPE iLO systems.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">2.</div><br>
    <p>Below is the set_ESKM_PrimaryKeyServer function. The function takes a Redfish client object, the primary key server address, and the primary key server port as input. It uses the Redfish API to configure the primary ESKM(Enterprise Secure Key Manager) key server on the iLO system.</p>
<br>It achieves this by the following steps:<br>

1. It first retrieves the relevant URI for the ESKM service by either using the resource directory or by navigating through the Redfish API endpoints.<br>
The function then constructs a request body with the primary key server address and port.
2. It sends a POST request to the ESKM URI with the request body to update the primary key server configuration.<br>
3. If the response is successful, it displays the updated configuration in JSON format.
If there are any errors or exceptions, it provides appropriate error messages or status codes.<br>
4. Finally, it logs out from the iLO server.
<br><br>
The set_ESKM_PrimaryKeyServer function enables the configuration of the primary ESKM key server for HPE iLO systems. It retrieves the relevant URI, sets the primary key server address and port, and updates the configuration via a POST request. It provides feedback on the success or failure of the operation and allows for proper error handling.
</div>

In [ ]:
def set_ESKM_PrimaryKeyServer(_redfishobj, primary_key_server_address, primary_key_server_port):

    eskm_uri = None
    body = dict()

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        security_service_uri = managers_members_response.obj.Oem.Hpe.Links['SecurityService']\
                                                                                    ['@odata.id']
        security_service_response = _redfishobj.get(security_service_uri)
        eskm_uri = security_service_response.obj.Links['ESKM']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeESKM.' in instance['@odata.type']:
                eskm_uri = instance['@odata.id']
                break

    if eskm_uri:

        body["PrimaryKeyServerAddress"] = primary_key_server_address
        body["PrimaryKeyServerPort"] = int(primary_key_server_port)

        resp = _redfishobj.post(eskm_uri, body)
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">3.</div><br>
    
The code below establishes a connection with an iLO server using the specified system URL, login account, and password. It then sets the primary ESKM key server address and port. The script utilizes the Redfish protocol to communicate with the server and performs the necessary operations to configure the key server. Finally, it logs out from the iLO server.
</div>

<div class = "alert alert-block alert-warning" style="border-radius: 20px;">
    <b>When running on the server locally</b> use the following values<br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    PRIMARY_KEY_SERVER_ADDRESS = "192.168.1.1"
    PRIMARY_KEY_SERVER_PORT = "9000"

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_ESKM_PrimaryKeyServer(REDFISHOBJ, PRIMARY_KEY_SERVER_ADDRESS, PRIMARY_KEY_SERVER_PORT)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>